In [1]:
#!/usr/bin/env python
# coding: utf8
"""A simple example of extracting relations between phrases and entities using
spaCy's named entity recognizer and the dependency parse. Here, we extract
money and currency values (entities labelled as MONEY) and then check the
dependency tree to find the noun phrase they are referring to – for example:
$9.4 million --> Net income.
Compatible with: spaCy v2.0.0+
Last tested with: v2.1.0
"""
from __future__ import unicode_literals, print_function

import plac
import spacy


TEXTS = [
    "Net income was $9.4 million compared to the prior year of $2.7 million.",
    "Revenue exceeded twelve billion dollars, with a loss of $1b.",
]



def main(model="en_core_web_sm"):
    nlp = spacy.load(model)
    print("Loaded model '%s'" % model)
    print("Processing %d texts" % len(TEXTS))

   

def filter_spans(spans):
    # Filter a sequence of spans so they don't contain overlaps
    get_sort_key = lambda span: (span.end - span.start, span.start)
    sorted_spans = sorted(spans, key=get_sort_key, reverse=True)
    result = []
    seen_tokens = set()
    for span in sorted_spans:
        if span.start not in seen_tokens and span.end - 1 not in seen_tokens:
            result.append(span)
            seen_tokens.update(range(span.start, span.end))
    return result


def extract_currency_relations(doc):
    # Merge entities and noun chunks into one token
    spans = list(doc.ents) + list(doc.noun_chunks)
    spans = filter_spans(spans)
    with doc.retokenize() as retokenizer:
        for span in spans:
            retokenizer.merge(span)

    relations = []
    for money in filter(lambda w: w.ent_type_ == "MONEY", doc):
        if money.dep_ in ("attr", "dobj"):
            subject = [w for w in money.head.lefts if w.dep_ == "nsubj"]
            if subject:
                subject = subject[0]
                relations.append((subject, money))
        elif money.dep_ == "pobj" and money.head.dep_ == "prep":
            relations.append((money.head.head, money))
    return relations





In [2]:
model="en_core_web_sm"
nlp = spacy.load(model)
for text in TEXTS:
        doc = nlp(text)
        relations = extract_currency_relations(doc)
        for r1, r2 in relations:
            print("{:<10}\t{}\t{}".format(r1.text, r2.ent_type_, r2.text))
    
    

Net income	MONEY	$9.4 million
the prior year	MONEY	$2.7 million
Revenue   	MONEY	twelve billion dollars
a loss    	MONEY	1b


In [3]:
doc=nlp('Net income was $9.4 million compared to the prior year of $2.7 million.')

In [4]:
doc.ents

($9.4 million, the prior year, $2.7 million)

In [5]:
doc.noun_chunks

In [6]:
for n in doc.noun_chunks:
    print(n)

Net income
the prior year


In [7]:
import nltk
verb = "<ADV>*<AUX>*<VBN><IN|PART>*<ADV>*"
word = "<NOUN|ADJ|ADV|DET|ADP>"
preposition = "<ADP|ADJ>"

rel_pattern = "( %s (%s* (%s)+ )? )+ " % (verb, word, preposition)
grammar_long = '''REL_PHRASE: {%s}''' % rel_pattern
reverb_pattern = nltk.RegexpParser(grammar_long)

#sent = "where the equation caused by the eccentricity is maximum."
sent= " kepler was born in Germany"
sent_pos_tags = nltk.tag.pos_tag(sent.split())

for x in reverb_pattern.parse(sent_pos_tags):
    if isinstance(x, nltk.Tree) and x.label() == 'REL_PHRASE':
        rel_phrase = " ".join([t[0] for t in x.leaves()])
        print(rel_phrase)

born in


In [8]:
sent_pos_tags

[('kepler', 'NN'),
 ('was', 'VBD'),
 ('born', 'VBN'),
 ('in', 'IN'),
 ('Germany', 'NNP')]

In [9]:
rel_phrase

'born in'

In [10]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)
pd.set_option('max_colwidth', 260)


In [11]:
df=pd.read_json('dfObsV01.json');
df=df.sort_index();
len(df)


6699

In [12]:
df.head(10)

,sents,chapter,aso,entities,fner,label
0,Chapter 1,1,{'act': 'Chapter'},[],"[0, 0, 0]",0
1,"On the distinction between the first motion and the second or proper motions; and in the proper motions, between the first and the second inequality.",1,{'act': 'On'},[],"[0, 0, 0]",0
2,The testimony of the ages confirms that the motions of the planets are orbicular.,1,"{'act': 'confirms', 'subject': 'testimony'}",[],"[0, 0, 0]",0
3,"Reason, having borrowed from experience, immediately presumes this: that their gyrations are perfect circles.",1,"{'act': 'presumes', 'subject': 'reason', 'obj': 'this: that their gyrations are perfect circles'}",[],"[0, 0, 0]",0
4,"For among figures it is circles, and among bodies the heavens, that are considered the most perfect.",1,{'act': 'considered'},[],"[0, 0, 0]",0
5,"However, when experience seems to teach something different to those who pay careful attention, namely, that the planets deviate from a simple circular path, it gives rise to a powerful sense of wonder, which at length drives people to look into causes.",1,"{'act': 'gives', 'subject': 'it', 'obj': 'rise '}","[[rise, ASTR]]","[0, 0, 0]",0
6,It is just this from which astronomy arose among humans.,1,"{'act': 'is', 'subject': 'it'}",[],"[0, 0, 0]",0
7,"Astronomy's aim is considered to be to show why the stars' motions appear to be irregular on earth, despite their being exceedingly well ordered in heaven, and to investigate the specific circles whereby the stars may be moved, so that by their aid the pos...",1,{'act': 'considered'},[],"[0, 0, 0]",0
8,"Before the distinction between the first motion(1) and the second motions(2) was established, people noted (in contemplating the sun, moon and stars) that their diurnal paths were visually very nearly equivalent to circles.",1,{'act': 'established'},"[[diurnal, ASTR]]","[0, 0, 0]",0
9,"These were, however, entwined one upon another like yarn on a ball, and the circles were for the most part smaller(3) circles of the sphere, rarely the greatest(4) (such",1,"{'act': 'were', 'subject': 'circles'}",[],"[0, 0, 0]",0


In [13]:
df.tail()

,sents,chapter,aso,entities,fner,label
6694,"To find the position of Mars at opposition, Kepler computed the angular distance that Mars and Earth—now substituting the place of the Sun—moved during 17 hours 20 minutes; Mars moved eastward about 16' 20"" and the Sun westward about 42' 18"".",appendix b,"{'act': 'moved', 'subject': 'mars'}","[[Mars, PLAN], [opposition, ASTR], [Mars, PLAN], [Earth, PLAN], [Sun, PLAN], [17 hours 20 minutes, TIME], [Mars, PLAN], [eastward, ASTR], [16' 20"", LONG], [Sun, PLAN], [42' 18"", LONG]]","[0, 1, 1]",0
6695,"Accordingly, Kepler determined the longitude of Mars at opposition to be 198° 37' 50"" from which he subtracted about 39"" in order to correct Mars's orbit; he got 198° 37' 10"" (18° 37' 10"" Libra).",appendix b,"{'act': 'got', 'subject': 'he', 'obj': '' 10'}","[[longitude, ASTR], [Mars, PLAN], [opposition, ASTR], [198° 37' 50"", LONG], [Mars, PLAN], [orbit, ASTR], [198° 37' 10"", LONG], [18° 37' 10"" Libra, LONG]]","[0, 0, 1]",0
6696,The Sun moved westward and its longitude decreased from the time of observation to its position opposite to Mars.,appendix b,"{'act': 'moved', 'subject': 'sun'}","[[Sun, PLAN], [longitude, ASTR], [observation, ASTR], [Mars, PLAN]]","[0, 0, 0]",0
6697,"Therefore, the time of opposition is 17 hours 20 minutes before March 29, at 21:43, the time when the observation was made.",appendix b,"{'act': 'is', 'subject': 'time'}","[[opposition, ASTR], [17 hours 20 minutes, TIME], [March 29, DATE], [observation, ASTR]]","[1, 1, 0]",0
6698,"Kepler determined the time of opposition on March 28, 4h 23m AM, old style.",appendix b,"{'act': 'determined', 'subject': 'kepler', 'obj': 'the time of opposition on March 28'}","[[opposition, ASTR], [March 28, DATE], [4h 23m, TIME]]","[1, 1, 0]",0


In [14]:
def reverb(sent):
    verb = "<ADV>*<AUX>*<VBN><IN|PART>*<ADV>*"
    word = "<NOUN|ADJ|ADV|DET|ADP>"
    preposition = "<ADP|ADJ>"

    rel_pattern = "( %s (%s* (%s)+ )? )+ " % (verb, word, preposition)
    grammar_long = '''REL_PHRASE: {%s}''' % rel_pattern
    reverb_pattern = nltk.RegexpParser(grammar_long)

    #sent = "where the equation caused by the eccentricity is maximum."
    #sent= " kepler was born in Germany"
    sent_pos_tags = nltk.tag.pos_tag(sent.split())

    for x in reverb_pattern.parse(sent_pos_tags):
        if isinstance(x, nltk.Tree) and x.label() == 'REL_PHRASE':
            rel_phrase = " ".join([t[0] for t in x.leaves()])
            return rel_phrase

In [15]:
df["relation"]=df["sents"].apply(lambda x: reverb(x)  )

In [16]:
df.head(40)

,sents,chapter,aso,entities,fner,label,relation
0,Chapter 1,1,{'act': 'Chapter'},[],"[0, 0, 0]",0,None
1,"On the distinction between the first motion and the second or proper motions; and in the proper motions, between the first and the second inequality.",1,{'act': 'On'},[],"[0, 0, 0]",0,None
2,The testimony of the ages confirms that the motions of the planets are orbicular.,1,"{'act': 'confirms', 'subject': 'testimony'}",[],"[0, 0, 0]",0,None
3,"Reason, having borrowed from experience, immediately presumes this: that their gyrations are perfect circles.",1,"{'act': 'presumes', 'subject': 'reason', 'obj': 'this: that their gyrations are perfect circles'}",[],"[0, 0, 0]",0,borrowed from
4,"For among figures it is circles, and among bodies the heavens, that are considered the most perfect.",1,{'act': 'considered'},[],"[0, 0, 0]",0,considered
5,"However, when experience seems to teach something different to those who pay careful attention, namely, that the planets deviate from a simple circular path, it gives rise to a powerful sense of wonder, which at length drives people to look into causes.",1,"{'act': 'gives', 'subject': 'it', 'obj': 'rise '}","[[rise, ASTR]]","[0, 0, 0]",0,None
6,It is just this from which astronomy arose among humans.,1,"{'act': 'is', 'subject': 'it'}",[],"[0, 0, 0]",0,None
7,"Astronomy's aim is considered to be to show why the stars' motions appear to be irregular on earth, despite their being exceedingly well ordered in heaven, and to investigate the specific circles whereby the stars may be moved, so that by their aid the pos...",1,{'act': 'considered'},[],"[0, 0, 0]",0,considered
8,"Before the distinction between the first motion(1) and the second motions(2) was established, people noted (in contemplating the sun, moon and stars) that their diurnal paths were visually very nearly equivalent to circles.",1,{'act': 'established'},"[[diurnal, ASTR]]","[0, 0, 0]",0,None
9,"These were, however, entwined one upon another like yarn on a ball, and the circles were for the most part smaller(3) circles of the sphere, rarely the greatest(4) (such",1,"{'act': 'were', 'subject': 'circles'}",[],"[0, 0, 0]",0,None


In [17]:
df

,sents,chapter,aso,entities,fner,label,relation
0,Chapter 1,1,{'act': 'Chapter'},[],"[0, 0, 0]",0,None
1,"On the distinction between the first motion and the second or proper motions; and in the proper motions, between the first and the second inequality.",1,{'act': 'On'},[],"[0, 0, 0]",0,None
2,The testimony of the ages confirms that the motions of the planets are orbicular.,1,"{'act': 'confirms', 'subject': 'testimony'}",[],"[0, 0, 0]",0,None
3,"Reason, having borrowed from experience, immediately presumes this: that their gyrations are perfect circles.",1,"{'act': 'presumes', 'subject': 'reason', 'obj': 'this: that their gyrations are perfect circles'}",[],"[0, 0, 0]",0,borrowed from
4,"For among figures it is circles, and among bodies the heavens, that are considered the most perfect.",1,{'act': 'considered'},[],"[0, 0, 0]",0,considered
...,...,...,...,...,...,...,...
6694,"To find the position of Mars at opposition, Kepler computed the angular distance that Mars and Earth—now substituting the place of the Sun—moved during 17 hours 20 minutes; Mars moved eastward about 16' 20"" and the Sun westward about 42' 18"".",appendix b,"{'act': 'moved', 'subject': 'mars'}","[[Mars, PLAN], [opposition, ASTR], [Mars, PLAN], [Earth, PLAN], [Sun, PLAN], [17 hours 20 minutes, TIME], [Mars, PLAN], [eastward, ASTR], [16' 20"", LONG], [Sun, PLAN], [42' 18"", LONG]]","[0, 1, 1]",0,None
6695,"Accordingly, Kepler determined the longitude of Mars at opposition to be 198° 37' 50"" from which he subtracted about 39"" in order to correct Mars's orbit; he got 198° 37' 10"" (18° 37' 10"" Libra).",appendix b,"{'act': 'got', 'subject': 'he', 'obj': '' 10'}","[[longitude, ASTR], [Mars, PLAN], [opposition, ASTR], [198° 37' 50"", LONG], [Mars, PLAN], [orbit, ASTR], [198° 37' 10"", LONG], [18° 37' 10"" Libra, LONG]]","[0, 0, 1]",0,None
6696,The Sun moved westward and its longitude decreased from the time of observation to its position opposite to Mars.,appendix b,"{'act': 'moved', 'subject': 'sun'}","[[Sun, PLAN], [longitude, ASTR], [observation, ASTR], [Mars, PLAN]]","[0, 0, 0]",0,decreased from
6697,"Therefore, the time of opposition is 17 hours 20 minutes before March 29, at 21:43, the time when the observation was made.",appendix b,"{'act': 'is', 'subject': 'time'}","[[opposition, ASTR], [17 hours 20 minutes, TIME], [March 29, DATE], [observation, ASTR]]","[1, 1, 0]",0,None


In [24]:
len(set(df.relation))

1208

In [23]:
set(df.relation)

{'assumed in',
 'joined,',
 'covered; between',
 'obstructed',
 'persuaded that',
 'confirmed in',
 'appropriate,',
 'certain,',
 'spread',
 'cast;',
 'observed that',
 'measured on',
 'bound',
 'returned',
 'see on',
 'displayed by',
 'confirmed',
 'been shown in',
 'proved, in',
 'made in',
 'shown that',
 'found in',
 'denied in',
 'explored above',
 'increased by',
 'extended over',
 'affected by',
 'been moved',
 'brought into',
 'hitherto required',
 'confirmed that',
 'said]',
 'increased at',
 'preceded by',
 'discrepant by',
 'increased, because',
 'elapsed',
 'had',
 'bisected by',
 'proved',
 'turned',
 'found as',
 'assumed among',
 'been observed',
 'seen as',
 'known through',
 'arrived at',
 'noticed that',
 'ferrymen, by',
 'perceived in',
 'separated from',
 'inferred',
 'constructed at',
 'parallel',
 'worked',
 'accomplished by',
 'proposed observations,',
 'been accomplished',
 'presented by',
 'multiplied by',
 'smaller, because',
 'determined that',
 'culminating,

In [26]:
df.relation.value_counts()

found                82
been                 76
given                62
made                 50
set                  42
                     ..
carried around on     1
θ,                    1
right) while          1
sent                  1
assumed in            1
Name: relation, Length: 1207, dtype: int64